In [1]:
import os
import rasterio
import numpy as np

def calculate_slope_rasterio(input_dem_path, output_slope_path, output_in_percent=False):
    """
    Calculates slope from a DEM using the Rasterio library.
    This is a more Python-native approach and avoids PATH issues with gdaldem.

    Args:
        input_dem_path (str): The full path to the input DEM file.
        output_slope_path (str): The full path to save the output slope raster file.
        output_in_percent (bool): If True, slope is calculated as percent.
                                  If False (default), slope is in degrees.
    """
    if not os.path.exists(input_dem_path):
        print(f"!!! Error: Input DEM file not found at '{input_dem_path}'.")
        return

    print(f"\n--- Starting Slope Calculation (using Rasterio) ---")
    print(f"Input DEM: {os.path.basename(input_dem_path)}")
    print(f"Output Slope File: {os.path.basename(output_slope_path)}")
    print(f"Output Units: {'Percent' if output_in_percent else 'Degrees'}")

    try:
        with rasterio.open(input_dem_path) as src:
            # Read the DEM data
            dem = src.read(1)
            # Copy the metadata from the source raster
            meta = src.meta.copy()

            # Get the cell size from the geotransform
            # Transform is (x_resolution, row_rotation, x_origin, col_rotation, y_resolution, y_origin)
            # x_resolution is meta['transform'][0]
            # y_resolution is meta['transform'][4] (it's often negative)
            x_res = meta['transform'][0]
            y_res = abs(meta['transform'][4])

            # Calculate the slope using NumPy's gradient function
            # This calculates the change in elevation in the y and x directions
            gradient_y, gradient_x = np.gradient(dem, y_res, x_res)
            
            # Calculate slope in radians
            slope_rad = np.arctan(np.sqrt(gradient_x**2 + gradient_y**2))

            if output_in_percent:
                # Convert slope from radians to percent rise
                slope_output = np.tan(slope_rad) * 100
            else:
                # Convert slope from radians to radians to degrees
                slope_output = np.rad2deg(slope_rad)

            # Update metadata for the output slope raster
            meta.update({
                'dtype': 'float32',
                'nodata': -9999, # Set a NoData value
                'compress': 'lzw'
            })

            # Write the slope data to a new GeoTIFF file
            with rasterio.open(output_slope_path, 'w', **meta) as dst:
                dst.write(slope_output.astype(rasterio.float32), 1)

            print(f"\nSuccessfully calculated slope and saved to: {output_slope_path}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: VERIFY YOUR INPUTS AND SETTINGS HERE ---

    # 1. Path to your final merged DEM file
    # This path should be correct if merge_rasters.ipynb has run successfully for DEM
    merged_dem_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final_UTM.tif" 

    # 2. Path to save the new slope raster file
    # This path will save the output in the Suitability_Layers subfolder
    slope_output_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/Slope_Degrees.tif"
    
    # Create the output directory if it doesn't exist
    output_dir = os.path.dirname(slope_output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 3. Choose output units for slope
    # False = Degrees (recommended for general analysis)
    # True = Percent
    calculate_as_percent = False

    # --- END OF USER SETTINGS ---

    # --- Run the Slope Calculation ---
    calculate_slope_rasterio(merged_dem_path, slope_output_path, calculate_as_percent)

    print("\nSlope calculation process finished.")


--- Starting Slope Calculation (using Rasterio) ---
Input DEM: DEM_Merged_Final_UTM.tif
Output Slope File: Slope_Degrees.tif
Output Units: Degrees

Successfully calculated slope and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/Slope_Degrees.tif

Slope calculation process finished.
